In [2]:
# code to convert EFT LHE file (uncompressed) to a ROOT file with physics objects and weights
import numpy as np
import ROOT
import array
import math

#input file path
lhefile = "/Users/johno.aremu/Documents/pp2l2vNP1/unweighted_events.lhe"
root_file_path = "pp2l2vNP1.root"

#open the LHE file
root_file = ROOT.TFile(root_file_path, "RECREATE")
tree = ROOT.TTree("Events", "pp --> ZZ --> 2l2v EFT events")

# Define branches for the tree
run = array.array('i', [1])
luminosityBlock = array.array('I', [1])
event_number = array.array('L', [0])

#Muons
nMuon = array.array('I', [0])
Muon_pt = ROOT.std.vector('float')()
Muon_eta = ROOT.std.vector('float')()
Muon_phi = ROOT.std.vector('float')()
Muon_mass = ROOT.std.vector('float')()
Muon_charge = ROOT.std.vector('int')()

#Electrons
nElectron = array.array('I', [0])
Electron_pt = ROOT.std.vector('float')()
Electron_eta = ROOT.std.vector('float')()
Electron_phi = ROOT.std.vector('float')()
Electron_mass = ROOT.std.vector('float')()
Electron_charge = ROOT.std.vector('int')()

#Neutrinos
nNeutrino = array.array('I', [0])
Neutrino_pt = ROOT.std.vector('float')()
Neutrino_eta = ROOT.std.vector('float')()
Neutrino_phi = ROOT.std.vector('float')()
Neutrino_mass = ROOT.std.vector('float')()

#MET
MET_pt = array.array('f', [0.])
MET_phi = array.array('f', [0.])

#EFT weights
weight_SM = array.array('f', [0.])
weight_EFT1 = array.array('f', [0.])
weight_EFT2 = array.array('f', [0.])
weight_EFT3 = array.array('f', [0.])
weight_EFT4 = array.array('f', [0.])
weight_EFT5 = array.array('f', [0.])

#Define branches in the tree
tree.Branch("run", run, "run/I")
tree.Branch("luminosityBlock", luminosityBlock, "luminosityBlock/i")
tree.Branch("event", event_number, "event/l")

#Muons
tree.Branch("nMuon", nMuon, "nMuon/i")
tree.Branch("Muon_pt", Muon_pt)
tree.Branch("Muon_eta", Muon_eta)
tree.Branch("Muon_phi", Muon_phi)
tree.Branch("Muon_mass", Muon_mass)
tree.Branch("Muon_charge", Muon_charge)

#Electrons
tree.Branch("nElectron", nElectron, "nElectron/i")
tree.Branch("Electron_pt", Electron_pt)
tree.Branch("Electron_eta", Electron_eta)
tree.Branch("Electron_phi", Electron_phi)
tree.Branch("Electron_mass", Electron_mass)
tree.Branch("Electron_charge", Electron_charge)

#Neutrinos
tree.Branch("nNeutrino", nNeutrino, "nNeutrino/i")
tree.Branch("Neutrino_pt", Neutrino_pt)
tree.Branch("Neutrino_eta", Neutrino_eta)
tree.Branch("Neutrino_phi", Neutrino_phi)
tree.Branch("Neutrino_mass", Neutrino_mass)

#MET
tree.Branch("MET_pt", MET_pt, "MET_pt/F")
tree.Branch("MET_phi", MET_phi, "MET_phi/F")

#EFT weights
tree.Branch("weight_SM", weight_SM, "weight_SM/F")
tree.Branch("weight_EFT1", weight_EFT1, "weight_EFT1/F")
tree.Branch("weight_EFT2", weight_EFT2, "weight_EFT2/F")
tree.Branch("weight_EFT3", weight_EFT3, "weight_EFT3/F")
tree.Branch("weight_EFT4", weight_EFT4, "weight_EFT4/F")
tree.Branch("weight_EFT5", weight_EFT5, "weight_EFT5/F")

# Function to compute pt, eta, phi from px, py, pz
def compute_pt_eta_phi(px, py, pz):
    pt = math.sqrt(px**2 + py**2)
    p = math.sqrt(px**2 + py**2 + pz**2)
    eta = 0.5 * math.log((p + pz) / (p - pz)) if p != abs(pz) else 0
    phi = math.atan2(py, px)
    return pt, eta, phi

# Read the LHE file and fill the tree
with open(lhefile, 'r') as f:
    in_event = False
    in_weights = False
    event_lines = []
    weight_lines = []
    event_id = 0

    for line in f:
        line = line.strip()

        if line == "<event>":
            in_event = True
            event_lines = []
            weight_lines = []
        elif line == "</event>":
            in_event = False
            event_id += 1
            event_number[0] = event_id

            # reset all vectors
            Muon_pt.clear(); 
            Muon_eta.clear(); 
            Muon_phi.clear(); 
            Muon_mass.clear(); 
            Muon_charge.clear()
            Electron_pt.clear(); 
            Electron_eta.clear(); 
            Electron_phi.clear(); 
            Electron_mass.clear(); 
            Electron_charge.clear()
            Neutrino_pt.clear(); 
            Neutrino_eta.clear(); 
            Neutrino_phi.clear(); 
            Neutrino_mass.clear()

            met_px = 0.0
            met_py = 0.0

            # parse particles
            for p in event_lines[1:]:
                tokens = p.split()
                if len(tokens) < 11:
                    continue

                pdg_id = int(tokens[0])
                px = float(tokens[6]); py = float(tokens[7]); pz = float(tokens[8])
                E = float(tokens[9]); mass = float(tokens[10])

                charge = 0
                if pdg_id in [11, 13]: charge = -1
                elif pdg_id in [-11, -13]: charge = 1

                pt, eta, phi = compute_pt_eta_phi(px, py, pz)

                if abs(pdg_id) == 11:
                    Electron_pt.push_back(pt)
                    Electron_eta.push_back(eta)
                    Electron_phi.push_back(phi)
                    Electron_mass.push_back(mass)
                    Electron_charge.push_back(charge)
                elif abs(pdg_id) == 13:
                    Muon_pt.push_back(pt)
                    Muon_eta.push_back(eta)
                    Muon_phi.push_back(phi)
                    Muon_mass.push_back(mass)
                    Muon_charge.push_back(charge)
                elif abs(pdg_id) in [12, 14, 16]:
                    Neutrino_pt.push_back(pt)
                    Neutrino_eta.push_back(eta)
                    Neutrino_phi.push_back(phi)
                    Neutrino_mass.push_back(mass)

                    met_px += px
                    met_py += py

            # parse weights
            weight_SM[0] = 0.0
            for i in range(1, 6):
                globals()[f"weight_EFT{i}"][0] = 0.0

            if weight_lines:
                w = [float(x.split(">")[1].split("<")[0]) for x in weight_lines if "<wgt" in x]
                if len(w) >= 1: weight_SM[0] = w[0]
                for i in range(1, min(6, len(w))):
                    globals()[f"weight_EFT{i}"][0] = w[i]

            # finalize counts
            nMuon[0] = Muon_pt.size()
            nElectron[0] = Electron_pt.size()
            nNeutrino[0] = Neutrino_pt.size()

            MET_pt[0] = np.sqrt(met_px**2 + met_py**2)
            MET_phi[0] = np.arctan2(met_py, met_px)

            tree.Fill()

        elif in_event:
            if line.startswith("<rwgt>"):
                in_weights = True
            elif line.startswith("</rwgt>"):
                in_weights = False
            elif in_weights:
                weight_lines.append(line)
            else:
                event_lines.append(line)

# write file
root_file.Write()
root_file.Close()
print(f"Finished writing ROOT file: {root_file_path}")
print(f"Total events processed: {event_id}")

Finished writing ROOT file: pp2l2vNP1.root
Total events processed: 10000


In [3]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import hist
import vector

vector.register_awkward()

file = uproot.open("pp2l2vNP1.root")
tree = file["Events"]

#check the branches in the tree
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
run                  | int32_t                  | AsDtype('>i4')
luminosityBlock      | uint32_t                 | AsDtype('>u4')
event                | uint64_t                 | AsDtype('>u8')
nMuon                | uint32_t                 | AsDtype('>u4')
Muon_pt              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_eta             | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_phi             | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_mass            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_charge          | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
nElectron            | uint32_t                 | AsDtype('>u4')
Electron_pt          | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Electron_eta         | std: